<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-&amp;-Inits" data-toc-modified-id="Imports-&amp;-Inits-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports &amp; Inits</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preparation</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Testing</a></span><ul class="toc-item"><li><span><a href="#Ignite-Testing" data-toc-modified-id="Ignite-Testing-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Ignite Testing</a></span></li><li><span><a href="#NLPBook-Testing" data-toc-modified-id="NLPBook-Testing-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>NLPBook Testing</a></span></li></ul></li><li><span><a href="#Inference" data-toc-modified-id="Inference-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Inference</a></span><ul class="toc-item"><li><span><a href="#Single-Inference" data-toc-modified-id="Single-Inference-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Single Inference</a></span></li><li><span><a href="#TopK-Inference" data-toc-modified-id="TopK-Inference-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>TopK Inference</a></span></li></ul></li></ul></div>

# Surname Classifier with MLP

Classifying surnames based on national origin.

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from pathlib import Path

In [3]:
from ignite.engine import Events, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [4]:
from surname.dataset import CNNDataset
# from surname.model import CNNClassifer
from surname.trainer import Trainer
from surname.args import cnn_args as args

In [5]:
path = Path('../data/surnames')

## Data Preparation

In [6]:
work_dir = path/'work_dir'
surnames_csv = path/args.proc_dataset_csv
vectorizer_path = work_dir/args.vectorizer_fname
cw_file = work_dir/args.cw_file

args

Namespace(batch_size=64, checkpointer_name='cnn_classifier', checkpointer_prefix='surname', cw_file='class_weights.pt', device='cuda:3', dropout_p=0.1, early_stopping_criteria=5, hidden_dim=100, learning_rate=0.001, model_dir='models', num_channels=256, num_epochs=100, proc_dataset_csv='surnames_with_splits.csv', raw_dataset_csv='surnames.csv', save_every=2, save_total=5, train_proportion=0.7, vectorizer_fname='cnn_vectorizer.json')

In [7]:
df = pd.read_csv(surnames_csv)
len(df)

10980

In [8]:
is_load = True

In [9]:
if not is_load:
  train_ds = ProjectDataset.load_data_and_create_vectorizer(df.loc[df['split'] == 'train'])
  train_ds.save_vectorizer(vectorizer_path)
  vectorizer = train_ds.get_vectorizer()
  class_counts = df['nationality'].value_counts().to_dict()
  sorted_counts = sorted(class_counts.items(), key=lambda x: vectorizer.nationality_vocab.lookup_token(x[0]))
  freq = [count for _, count in sorted_counts]
  class_weights = 1.0/torch.tensor(freq, dtype=torch.float32)
  torch.save(class_weights, cw_file)

In [10]:
train_df = df.loc[df['split'] == 'train']
train_ds = CNNDataset.load_data_and_vectorizer(train_df, vectorizer_path)
vectorizer = train_ds.get_vectorizer()
class_weights = torch.load(cw_file)
train_dl = DataLoader(train_ds, args.batch_size, shuffle=True, drop_last=True)

val_df = df.loc[df['split'] == 'val']
val_ds = CNNDataset.load_data_and_vectorizer(val_df, vectorizer_path)
val_dl = DataLoader(val_ds, args.batch_size, shuffle=True, drop_last=True)

test_df = df.loc[df['split'] == 'test']
test_ds = CNNDataset.load_data_and_vectorizer(test_df, vectorizer_path)
test_dl = DataLoader(test_ds, args.batch_size, shuffle=True, drop_last=True)

len(train_dl.dataset), len(val_dl.dataset), len(test_dl.dataset)

(7680, 1640, 1660)

In [11]:
class CNNClassifier(nn.Module):
  def __init__(self, initial_num_channels, num_classes, num_channels):
    super(CNNClassifier, self).__init__()

    self.convnet = nn.Sequential(
          nn.Conv1d(in_channels=initial_num_channels, out_channels=num_channels, kernel_size=3),
          nn.ELU(),
          nn.Conv1d(in_channels=initial_num_channels, out_channels=num_channels, kernel_size=3,
            stride=2),
          nn.ELU(),
          nn.Conv1d(in_channels=initial_num_channels, out_channels=num_channels, kernel_size=3,
            stride=2),
          nn.ELU(),
          nn.Conv1d(in_channels=initial_num_channels, out_channels=num_channels, kernel_size=3,
            stride=2),
          nn.ELU()
        )
    self.fc = nn.Linear(num_channels, num_classes)
    self.softmax = nn.Softmax()

  def forward(self, x_in, apply_softmax=False):
    y_out = self.convnet(x_in).squeeze(dim=2)
    y_out = self.fc(y_out)

    if apply_softmax:
      y_out = self.softmax(y_out)

    return y_out

## Model

In [ ]:
classifier = CNNClassifier(initial_num_channels=len(vectorizer.surname_vocab), num_classes=len(vectorizer.nationality_vocab), num_channels=args.num_channels)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
class_weights = class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(class_weights)

In [13]:
pbar = ProgressBar(persist=True)
metrics = {'accuracy': Accuracy(), 'loss': Loss(loss_func)}

## Training

In [ ]:
surname_trainer = Trainer(classifier, optimizer, loss_func, train_dl, val_dl, work_dir, args, pbar, metrics)
surname_trainer.run()

## Testing

### Ignite Testing

In [ ]:
args.device = 'cpu'
classifier = Classifier(input_dim=len(vectorizer.surname_vocab), hidden_dim=args.hidden_dim,\
                        output_dim=len(vectorizer.nationality_vocab))
# state_dict = torch.load(work_dir/args.model_dir/'surname_classifier_24.pth')
state_dict = torch.load(work_dir/'surname_classifier_model.pth')
classifier.load_state_dict(state_dict)

class_weights = class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(class_weights)
metrics = {'accuracy': Accuracy(), 'loss': Loss(loss_func)}

In [ ]:
evaluator = create_supervised_evaluator(classifier, metrics=metrics)

@evaluator.on(Events.COMPLETED)
def log_testing_results(engine):
  metrics = engine.state.metrics
  print(f"Test loss: {metrics['loss']:0.3f}")
  print(f"Test accuracy: {metrics['accuracy']:0.3f}")

In [ ]:
evaluator.run(test_dl)

### NLPBook Testing

In [ ]:
def compute_accuracy(y_pred, y_target):
  _, y_pred_indices = y_pred.max(dim=1)
  n_correct = torch.eq(y_pred_indices, y_target).sum().item()
  return n_correct / len(y_pred_indices) * 100

In [ ]:
running_loss = 0.
running_acc = 0.

classifier.eval()
for i, batch in enumerate(test_dl):
  x,y = batch
  y_pred = classifier(x_in=x.float())
  
  loss = loss_func(y_pred, y)
  loss_t = loss.item()
  running_loss += (loss_t-running_loss)/(i+1)
  
  acc_t = compute_accuracy(y_pred, y)
  running_acc += (acc_t-running_acc)/(i+1)

In [ ]:
print(f"Test loss: {running_loss:0.3f}")
print(f"Test acc: {running_acc:0.3f}")

## Inference

### Single Inference

In [ ]:
def predict_natinoality(surname, classifier, vectorizer):
  """
    Predict the nationality from a new surname
    
    Args:
      surname: the surname to classify
      classifier: an instance of the classifier
      vectorizer: the corresponding vectorizer
      
    Returns:
      a dictionary with most likely natinoality and its probability
  """
  vectorized_surname = vectorizer.vectorize(surname)
  vectorized_surname = torch.tensor(vectorized_surname).view(1,-1)
  result = classifier(vectorized_surname, apply_softmax=True)
  
  probability_values, indices = result.max(dim=1)
  idx = indices.item()
  
  predicted_nationality = vectorizer.nationality_vocab.lookup_idx(idx)
  probability_value = probability_values.item()
  
  return {'nationality': predicted_nationality, 'probability': probability_value}

In [ ]:
new_surname = input("Enter a surname to classify: ")
prediction = predict_natinoality(new_surname, classifier, vectorizer)
print(f"{new_surname} -> {prediction['nationality']} p={prediction['probability']:0.2f}")

### TopK Inference

In [ ]:
def predict_topk_nationality(name, classifier, vectorizer, k=5):
  vectorized_name = vectorizer.vectorize(name)
  vectorized_name = torch.tensor(vectorized_name).view(1, -1)
  prediction_vector = classifier(vectorized_name, apply_softmax=True)
  probability_values, indices = torch.topk(prediction_vector, k=k)

  # returned size is 1,k
  probability_values = probability_values.detach().numpy()[0]
  indices = indices.detach().numpy()[0]

  results = []
  for prob_value, idx in zip(probability_values, indices):
      nationality = vectorizer.nationality_vocab.lookup_idx(idx)
      results.append({'nationality': nationality, 
                      'probability': prob_value})

  return results

In [ ]:
new_surname = input("Enter a surname to classify: ")
classifier = classifier.to("cpu")

k = int(input("How many of the top predictions to see? "))
if k > len(vectorizer.nationality_vocab):
  print("Sorry! That's more than the # of nationalities we have.. defaulting you to max size :)")
  k = len(vectorizer.nationality_vocab)
    
predictions = predict_topk_nationality(new_surname, classifier, vectorizer, k=k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
  print(f"{new_surname} -> {prediction['nationality']} p={prediction['probability']:0.2f}")